Для работы необходимо скачать датасет **archive.zip**:

https://www.kaggle.com/pranavraikokte/airplanes-dataset-for-rcnn?select=Airplanes_Annotations

И модель **Comp_Vision_Task_3_model.zip**:

https://de.ifmo.ru/nextcloud/index.php/s/KtcwoaaJTSQC9WD?path=%2F%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5%203

Всё закинуть в папку data без распаковки.

In [86]:
import zipfile
import shutil
import os

import tensorflow as tf
import cv2
import numpy as np
import cupy as cp
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import sklearn.metrics

use_graph = False

Для начала нужно распаковать датасет, благо он небольшой.

In [87]:
zip_file = "data/archive.zip"
try:
    shutil.rmtree("data/temp")
except:
    pass
os.mkdir("data/temp")
with zipfile.ZipFile(zip_file, 'r') as z:
    for file in z.namelist():
        z.extract(file, "data/temp")

Для выполнения заданий, описанных ниже, используйте предварительно обученную модель из рассмотренного примера для 
датасета с самолетами.

###### 1. Для следующего изображения (42847.jpg из датасета с самолетами) выполните сегментацию посредством метода 
createSelectiveSearchSegmentation() из библиотеки cv2 (используйте switchToSelectiveSearchFast()).

In [88]:
filename = "42847.jpg"
image = cv2.imread(f"data/temp/Images/Images/{filename}")

In [89]:
if use_graph:
    figure(figsize=(16, 9), dpi=80)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.xticks([]),plt.yticks([])
    print(filename)
    plt.show()

In [90]:
cv2.setUseOptimized(True)
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(image)
ss.switchToSelectiveSearchFast()
rects = ss.process()

In [91]:
if use_graph:
    figure(figsize=(16, 9), dpi=80)
    image_to_show = image.copy()
    for rect in rects:
        x, y, w, h = rect
        cv2.rectangle(image_to_show, (x, y), (x + w, y + h), (0, 255, 0), 1, cv2.LINE_AA)
    plt.imshow(cv2.cvtColor(image_to_show, cv2.COLOR_BGR2RGB))
    plt.xticks([]),plt.yticks([])
    print(filename)
    plt.show()

Сама модель огромная. Её нужно распаковать и загрузить веса.

In [92]:
zip_file = "data/Comp_Vision_Task_3_model.zip"
try:
    shutil.rmtree("data/temp")
except:
    pass
os.mkdir("data/temp")
with zipfile.ZipFile(zip_file, 'r') as z:
    for file in z.namelist():
        z.extract(file, "data/temp")

In [93]:
# Считывание модели
model = tf.keras.models.model_from_json(open('data/temp/Comp_Vision_Task_3_model.json').read())
model.load_weights('data/temp/ieeercnn_vgg16_1.h5')
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

Идея происходящего дальше:

1. Мы ищем с помощью SelectiveSearchSegmentation все возможные "объекты"

2. Каждый объект прогоняем через предобученную VGG16, и смотрим, с какой вероятностью объект относится
к классифициируемому классу. Идея близка к идее "скользящего окна", но только прямоугольники берутся не все, а
как результат сегментации cv2. 

3. VGG16 - "Very Deep Convolutional Networks for Large-Scale Image Recognition" - нейронная сеть для классификации,
следовательно мы можем получить вероятность принадлежности объекта на данном изображении к обученному классу.
Производится фильтрация всех прямоугольников по этому порогу threshold и сохраняются наиболее вероятные объекты, 
принадлежащие к требуемому классу.

In [98]:
threshold = 0.65
rects_result = []

image_to_show = image.copy()
for index, rect in enumerate(rects):
    x, y, w, h = rect
    rect_image = image[y: y + h,x: x + w]
    rect_image = cv2.resize(rect_image, (224, 224), interpolation = cv2.INTER_AREA)
    rect_image = np.expand_dims(rect_image, axis = 0)
    rect_image = model.predict(rect_image)
    if rect_image[0][0] > threshold:
        cv2.rectangle(image_to_show, (x, y), (x + w, y + h), (0, 255, 0), 1, cv2.LINE_AA)
        rects_result.append([rect_image[0][0], rect])

predictions = np.array(rects_result)

[[0.81334037 array([90, 71, 24, 26], dtype=int32)]
 [0.914979 array([83, 71, 34, 27], dtype=int32)]
 [0.7603696 array([89, 71, 25, 27], dtype=int32)]
 [0.84444875 array([223, 153,  33,  35], dtype=int32)]
 [0.80786973 array([78, 71, 36, 26], dtype=int32)]
 [0.8054499 array([226, 155,  30,  33], dtype=int32)]
 [0.66221493 array([78, 71, 36, 46], dtype=int32)]
 [0.85726094 array([222, 154,  34,  34], dtype=int32)]
 [0.849152 array([82, 71, 32, 27], dtype=int32)]]


d:\programs\python\python37\lib\site-packages\ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [95]:
if use_graph:
    figure(figsize=(16, 9), dpi=80)
    plt.imshow(cv2.cvtColor(image_to_show, cv2.COLOR_BGR2RGB))
    plt.xticks([]),plt.yticks([])
    print(filename)
    plt.show()

###### 2. При помощи модели выполните обнаружение самолетов на изображении. Считать, что модель обнаружила самолет, если вероятность отнесения к классу "самолет" превышает 0.65.

Отдельно отметим, что модель может ошибаться и выделять в качестве самолетов части изображений, 
которые самолетов и не содержат. В любом случае, выделенные моделью области будем называть "самолетами".

Введите количество "самолетов", обнаруженных моделью:

In [96]:
print(f"Количество \"самолетов\", обнаруженных моделью: {len(rects_result)}")

Количество "самолетов", обнаруженных моделью: 9


###### 3. Среди выбранных "самолетов" определите тот, вероятность отнесения к классу "самолет" которого оказалась наибольшей.

In [99]:
predictions_max = predictions[predictions[:,0] == np.max(predictions[:,0])][0]

x, y, w, h = predictions_max[1]

print(f"Вероятность отнесения к классу \"самолет\" для этого предсказания: {predictions_max[0]}")

print(f"Координата x левого верхнего угла соответствующего прямоугольника: {x}")
print(f"Координата y левого верхнего угла соответствующего прямоугольника: {y}")
print(f"Ширина (в пикселях) соответствующего прямоугольника: {w}")
print(f"Высота (в пикселях) соответствующего прямоугольника: {h}")

Вероятность отнесения к классу "самолет" для этого предсказания: 0.9149789810180664
Координата x левого верхнего угла соответствующего прямоугольника: 83
Координата y левого верхнего угла соответствующего прямоугольника: 71
Ширина (в пикселях) соответствующего прямоугольника: 34
Высота (в пикселях) соответствующего прямоугольника: 27


###### 4. Среди выбранных "самолетов" определите тот, вероятность отнесения к классу "самолет" которого оказалась наименьшей.

In [100]:
predictions_min = predictions[predictions[:,0] == np.min(predictions[:,0])][0]

x, y, w, h = predictions_min[1]

print(f"Вероятность отнесения к классу \"самолет\" для этого предсказания: {predictions_min[0]}")

print(f"Координата x левого верхнего угла соответствующего прямоугольника: {x}")
print(f"Координата y левого верхнего угла соответствующего прямоугольника: {y}")
print(f"Ширина (в пикселях) соответствующего прямоугольника: {w}")
print(f"Высота (в пикселях) соответствующего прямоугольника: {h}")


Вероятность отнесения к классу "самолет" для этого предсказания: 0.6622149348258972
Координата x левого верхнего угла соответствующего прямоугольника: 78
Координата y левого верхнего угла соответствующего прямоугольника: 71
Ширина (в пикселях) соответствующего прямоугольника: 36
Высота (в пикселях) соответствующего прямоугольника: 46
